In [1]:
import pandas as pd
from scripts.lmfit_model import division_model, sum_model
from pathlib import Path
from tqdm.notebook import tqdm
import numpy as np
from sklearn.metrics import root_mean_squared_error
import matplotlib.pyplot as plt

### Read discharge and convert it to QMS

In [2]:
res_folder = Path("ConvertedQH/Discharge")
res_folder.mkdir(exist_ok=True, parents=True)

for gauge_id in tqdm(list(i.stem for i in Path("PreparedCurveData/InitialDischarge/").glob("*.txt"))):
    try:
        table_discharge = pd.read_csv(
            f"PreparedCurveData/InitialDischarge/{gauge_id}.txt", comment="#", sep="\t+", engine="python"
        )

    except (FileNotFoundError, pd.errors.EmptyDataError):
        continue
    if table_discharge.empty:
        continue
    table_discharge = table_discharge.drop(0).reset_index(drop=True)
    table_discharge["datetime"] = pd.to_datetime(table_discharge["datetime"])
    table_discharge = table_discharge.set_index("datetime")
    # Get the current column names
    columns = table_discharge.columns
    # Create a dictionary for renaming the last two columns
    new_column_names = {
        columns[-2]: "discharge",
        columns[-1]: "qc",
        "site_no": "gauge_id",
    }
    table_discharge.rename(columns=new_column_names, inplace=True)
    table_discharge = table_discharge.sort_index()

    table_discharge["discharge"] = table_discharge["discharge"].fillna(value=np.nan)
    table_discharge["discharge"] = pd.to_numeric(table_discharge["discharge"], errors="coerce")
    table_discharge["discharge"] = table_discharge["discharge"] * 0.028316832
    try:
        table_discharge = table_discharge.loc[pd.date_range(start="01-01-2000", end="12-31-2023"), :]
        table_discharge.index.name = "date"
        table_discharge.to_csv(f"{res_folder}/{gauge_id}.csv")
    except KeyError:
        continue


  0%|          | 0/1252 [00:00<?, ?it/s]

In [3]:
len(list(i.stem for i in Path(f"{res_folder}").glob("*.csv")))

235

### Read field measurments and store it for further curve approximation

In [4]:
res_folder = Path("ConvertedQH/DischargeLvL")
res_folder.mkdir(exist_ok=True, parents=True)

for gauge_id in tqdm(list(i.stem for i in Path("PreparedCurveData/InitialDischarge/").glob("*.txt"))):
    
    curve_file = pd.read_csv(f"PreparedCurveData/InitialQH/{gauge_id}.txt", sep="\t", comment="#")
    if curve_file.empty:
        continue
    curve_file = curve_file[["site_no", "measurement_dt", "gage_height_va", "discharge_va"]]
    curve_file = curve_file.dropna().drop(0).reset_index(drop=True)
    curve_file["measurement_dt"] = pd.to_datetime(curve_file["measurement_dt"])
    curve_file = curve_file.set_index("measurement_dt")
    curve_file[["gage_height_va", "discharge_va"]] = curve_file[["gage_height_va", "discharge_va"]].astype(
        float
    )
    curve_file["gage_height_va"] *= 0.3
    curve_file["discharge_va"] *= 0.028316832

    curve_file.rename(
        columns={
            "site_no": "gauge_id",
            "gage_height_va": "lvl_m",
            "discharge_va": "q_cms",
        },
        inplace=True,
    )

    curve_file.index.name = "date"
    curve_file = curve_file.loc["2000":"2023", :]
    curve_file.to_csv(f"{res_folder}/{gauge_id}.csv")
len(list(i.stem for i in Path(f"{res_folder}").glob("*.csv")))

  0%|          | 0/1252 [00:00<?, ?it/s]

679

### Prepare Q_H curve with lmfit

In [5]:
res_folder = Path("ConvertedQH/FitParams")
res_folder.mkdir(exist_ok=True, parents=True)

counter = 0
for gauge_id in tqdm(list(i.stem for i in Path("ConvertedQH/DischargeLvL").glob("*.csv"))):
    field_data = pd.read_csv(
        f"ConvertedQH/DischargeLvL/{gauge_id}.csv",
        index_col="date",
        parse_dates=True,
        dtype={"gauge_id": str},
    )
    field_data = field_data.sort_values(by="q_cms")
    field_data.loc[field_data["lvl_m"] < 0, "lvl_m"] = 0
    if len(field_data) < 4:
        continue
    y = field_data["lvl_m"].to_numpy()
    x = field_data["q_cms"].to_numpy()

    try:
        use_model = None
        use_model = division_model
        pars = use_model.make_params(
            power={"min": 0.001, "value": 0.1},
            c={"value": 0},
            slope={"value": 1, "min": 0},
            b={"value": 0},
        )
        result = use_model.fit(y, pars, x=x)
        error_res = root_mean_squared_error(y, result.best_fit)
        if error_res < 0.5:
            with open(f"ConvertedQH/FitParams/{gauge_id}_div.json", "w") as f:
                result.params.dump(f)
        else:
            counter += 1
            print(f"Bullshit ! for Gauge with ID {gauge_id} error is {error_res}")
    except ValueError:
        use_model = None
        use_model = sum_model
        pars = use_model.make_params(
            power={"min": 0.001, "value": 0.1},
            c={"value": 0},
            slope={"value": 1, "min": 0},
            b={"value": 0},
        )
        result = use_model.fit(y, pars, x=x)

        error_res = root_mean_squared_error(y, result.best_fit)
        if error_res < 0.5:
            with open(f"{res_folder}/{gauge_id}_sum.json", "w") as f:
                result.params.dump(f)
        else:
            counter += 1
            print(f"Bullshit ! for Gauge with ID {gauge_id} error is {error_res}")
        plt.plot(x, result.best_fit, "-", label="best fit")
        plt.plot(x, y, "*", label="field", color="green")
        plt.savefig(f"ConvertedQH/Images/{gauge_id}.png")
        plt.close()

print(
    f"Shit data for {counter} gauges out from {len(list(i.stem for i in Path('ConvertedQH/DischargeLvL').glob('*.csv')))}"
)
len(list(i.stem for i in Path(f"{res_folder}").glob("*.json")))

  0%|          | 0/679 [00:00<?, ?it/s]

Bullshit ! for Gauge with ID 10257600 error is 0.9617074523956122


/home/dmbrmv/miniconda3/envs/geo/lib/python3.10/site-packages/lmfit/model.py:1288: RuntimeWarning: invalid value encountered in divide
  return self.op(self.left.eval(params=params, **kwargs),


Bullshit ! for Gauge with ID 11156500 error is 0.8400415934262336
Bullshit ! for Gauge with ID 10270940 error is 0.5216639619265129
Bullshit ! for Gauge with ID 11140585 error is 0.688318831654689
Bullshit ! for Gauge with ID 11118500 error is 0.6932164805742201
Bullshit ! for Gauge with ID 10259050 error is 0.5354108519912992
Bullshit ! for Gauge with ID 11193030 error is 4.661373887438475
Bullshit ! for Gauge with ID 09429130 error is 0.6496299106252503
Bullshit ! for Gauge with ID 09429190 error is 0.5600051135972403
Bullshit ! for Gauge with ID 103087892 error is 1.5694436817457624
Bullshit ! for Gauge with ID 103087891 error is 0.5016997505512739
Bullshit ! for Gauge with ID 09429000 error is 2.3548395029945066
Bullshit ! for Gauge with ID 09523200 error is 3.408315496512111
Bullshit ! for Gauge with ID 11045700 error is 0.5648564663577931
Bullshit ! for Gauge with ID 10254730 error is 9.852521135470305
Bullshit ! for Gauge with ID 10270877 error is 0.5710572214358807
Bullshit ! f

0

### Create LvL series with model parameters based on discharges

In [33]:
bad_visual = list(
    i.replace('"', "").replace(" ", "") for i in pd.read_csv("configs/bad_gauges.csv").columns
)
prepared_params = list(i.stem for i in Path("ConvertedQH/FitParams/").glob("*.json"))
param_gauges = [i.split("_") for i in prepared_params if i.split("_")[0] not in bad_visual]
q_gauges = list(i.stem for i in Path("ConvertedQH/Discharge/").glob("*.csv"))
param_gauges = [i for i in param_gauges if i[0] in q_gauges]

In [70]:
res_folder = Path("ConvertedQH/ResultQH")
res_folder.mkdir(exist_ok=True, parents=True)

for desc in tqdm(param_gauges):
    result = None
    gauge_id, model_type = desc
    table_discharge = pd.read_csv(
        f"ConvertedQH/Discharge/{gauge_id}.csv",
        index_col="date",
        parse_dates=True,
        dtype={"gauge_id": str, "discharge": float},
    )
    qh_table = pd.read_csv(
        f"ConvertedQH/DischargeLvL/{gauge_id}.csv",
        index_col="date",
        parse_dates=True,
        dtype={"gauge_id": str, "q_cms": float, "lvl_m": float},
    )
    y = qh_table["lvl_m"].to_numpy()
    x = qh_table["q_cms"].to_numpy()
    if model_type == "sum":
        pars = sum_model.make_params(
            power={"min": 0.001, "value": 0.1},
            c={"value": 0},
            slope={"value": 1, "min": 0},
            b={"value": 0},
        )
        result = sum_model.fit(y, pars, x=x)

    elif model_type == "div":
        pars = division_model.make_params(
            power={"min": 0.001, "value": 0.1},
            c={"value": 0},
            slope={"value": 1, "min": 0},
            b={"value": 0},
        )
        result = division_model.fit(y, pars, x=x)
    else:
        print(f"{gauge_id}")

    with open(f"ConvertedQH/FitParams/{gauge_id}_{model_type}.json", "r") as f:
        params = result.params.load(f)

    table_discharge["lvl"] = [
        result.eval(x=i, params=params) for i in table_discharge["discharge"].to_numpy()
    ]
    table_discharge.to_csv(f"{res_folder}/{gauge_id}.csv")
    # plt.plot(x, y, "*", label="field", color="green")
    # plt.plot(
    #     table_discharge["discharge"], table_discharge["lvl"], "*", label="predictions", color="red"
    # )
    # plt.legend()
    # plt.savefig(f"ConvertedQH/ImagesQH/{gauge_id}.png")
    # plt.close()

len(list(i.stem for i in Path(f"{res_folder}").glob("*.csv")))


  0%|          | 0/164 [00:00<?, ?it/s]

164

### Prepare geodataframe with final gauges

In [2]:
import geopandas as gpd
from shapely.geometry import Point
import pandas as pd

initial_points = pd.read_csv("data/NWISMapperExport.csv",
                             dtype={"SiteNumber": str}, index_col="SiteNumber")

initial_points.head()

,SiteName,SiteCategory,SiteAgency,SiteLongitude,SiteLatitude,SiteNWISURL
SiteNumber,,,,,,
09423350,CARUTHERS C NR IVANPAH CA,ST,USGS,-115.298989,35.245006,https://waterdata.usgs.gov/nwis/inventory?agen...
09424050,CHEMEHUEVI WASH TRIB NR NEEDLES CA,ST,USGS,-114.603574,34.508343,https://waterdata.usgs.gov/nwis/inventory?agen...
09424150,"COLORADO RIVER AQUEDUCT NEAR PARKER DAM, AZ-CA",ST,USGS,-114.157170,34.316126,https://waterdata.usgs.gov/nwis/inventory?agen...
09424170,GENE WASH BL GENE WASH RES N PARKER DAM AZ-CAL,ST,USGS,-114.163281,34.300015,https://waterdata.usgs.gov/nwis/inventory?agen...
09427520,"COLORADO RIVER BELOW PARKER DAM, AZ-CA",ST,USGS,-114.140225,34.295570,https://waterdata.usgs.gov/nwis/inventory?agen...


In [16]:
my_custom_points = ["11143200", "11152500", "11152650", "1117777", "11159200"]

points = gpd.read_file("/mnt/c/Users/dmbrmv/DevWindows/Skoltech/california_flood/gauge_points.gpkg")

points = points.loc[points.SiteNumber.isin(my_custom_points), :].reset_index(drop=True)
points.to_file("data/custom_points.gpkg")

In [18]:
points


,SiteNumber,SiteName,SiteCategory,SiteAgency,SiteLongitude,SiteLatitude,SiteNWISURL,geometry
0,11159200,CORRALITOS C A FREEDOM CA,ST,USGS,-121.770506,36.939397,https://waterdata.usgs.gov/nwis/inventory?agen...,POINT (-121.77051 36.9394)
1,11152650,RECLAMATION DITCH NR SALINAS CA,ST-DCH,USGS,-121.704948,36.704959,https://waterdata.usgs.gov/nwis/inventory?agen...,POINT (-121.70495 36.70496)
2,11143200,CARMEL R A ROBLES DEL RIO CA,ST,USGS,-121.728889,36.474861,https://waterdata.usgs.gov/nwis/inventory?agen...,POINT (-121.72889 36.47486)
3,11152500,SALINAS R NR SPRECKELS CA,ST,USGS,-121.672446,36.631071,https://waterdata.usgs.gov/nwis/inventory?agen...,POINT (-121.67245 36.63107)
4,1117777,"Pajaro River, Watsonville",None,None,-121.751388,36.905046,None,POINT (-121.75139 36.90505)


In [73]:
final_gauges = list(i.stem for i in Path("ConvertedQH/ResultQH/").glob("*.csv"))
final_points = initial_points.loc[final_gauges, :]

final_points["geometry"] = [
    Point(lat, lon) for lat, lon in final_points[[" SiteLongitude", " SiteLatitude"]].values
]
final_points = gpd.GeoDataFrame(final_points, crs="EPSG:4326", geometry="geometry")

final_points.to_file("data/gauge_points.gpkg")
final_points.head()

,SiteName,SiteCategory,SiteAgency,SiteLongitude,SiteLatitude,SiteNWISURL,geometry
SiteNumber,,,,,,,
11161000,SAN LORENZO R A SANTA CRUZ CA,ST,USGS,-122.031908,36.990783,https://waterdata.usgs.gov/nwis/inventory?agen...,POINT (-122.03191 36.99078)
11133000,SANTA YNEZ R A NARROWS NR LOMPOC CA,ST,USGS,-120.424606,34.636095,https://waterdata.usgs.gov/nwis/inventory?agen...,POINT (-120.42461 34.63609)
10258500,PALM CYN C NR PALM SPRINGS CA,ST,USGS,-116.535571,33.745022,https://waterdata.usgs.gov/nwis/inventory?agen...,POINT (-116.53557 33.74502)
11042400,TEMECULA C NR AGUANGA CA,ST,USGS,-116.923636,33.459198,https://waterdata.usgs.gov/nwis/inventory?agen...,POINT (-116.92364 33.4592)
11152300,SALINAS R NR CHUALAR CA,ST,USGS,-121.549386,36.553573,https://waterdata.usgs.gov/nwis/inventory?agen...,POINT (-121.54939 36.55357)
